In [1]:
!pip install -Uqqq pip
!pip install bitsandbytes==0.43.0
!pip install -qqq torch==2.0.1
!pip install -qqq -U git+https://github.com/huggingface/transformers.git
!pip install -qqq -U git+https://github.com/huggingface/peft.git
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git
!pip install -qqq datasets==2.14.6
!pip install -qqq loralib==0.1.1
!pip install -qqq einops==0.6.1

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [19]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

import warnings
warnings.filterwarnings('ignore')

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [4]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.
You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [5]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
model.print_trainable_parameters

<bound method PeftModel.print_trainable_parameters of PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): FalconForCausalLM(
      (transformer): FalconModel(
        (word_embeddings): Embedding(65024, 4544)
        (h): ModuleList(
          (0-31): 32 x FalconDecoderLayer(
            (self_attention): FalconAttention(
              (maybe_rotary): FalconRotaryEmbedding()
              (query_key_value): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4544, out_features=4672, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4544, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4672, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lor

In [6]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [8]:
def generate_prompt(data_point):
  return f'''Imagine you are an interviewer interviewing a data science candidate and you are trying to give feedback based on the candidate's answer on the technical interview. Make sure to provide constructive feedback, answer in second person
The question you asked: {data_point["Question"]}
The candidate's Answer: {data_point["Answer"]}
Your feedback: '''.strip()

# def generate_prompt(data_point):
#   return f'''You are an interviewer evaluating a data science interviewee. You want to give constructive feedback directly to the interviewee. Identify the correctness of the answer and you aim to help the interviewee understand where they can improve. Make sure your feedback is in the second person (using "you") and does not refer to the interviewee as "the candidate."
# The question you asked: {data_point["Question"]}
# The candidate's Answer: {data_point["Answer"]}
# Your feedback: '''

In [9]:
def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_prompt

In [23]:
data = load_dataset("csv", data_files="/content/feedback_dataset_cleaned.csv")['train']

In [26]:
data = data.shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/1777 [00:00<?, ? examples/s]

In [28]:
split_data = data.train_test_split(test_size=0.1)

train_data = split_data['train']
eval_data = split_data['test']

In [ ]:
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=1e-5,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir="model",
    evaluation_strategy="steps",
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    load_best_model_at_end=True
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

model.config.use_cache = False
trainer.train()

Step,Training Loss,Validation Loss
1,2.902000,2.868520
2,2.973800,2.867882
3,2.795800,2.867757
4,2.605000,2.866715
5,2.741100,2.865844
6,2.889400,2.864560


In [ ]:
model.save_pretrained("fine-tuned")

In [ ]:
notebook_login()

In [ ]:
PEFT_MODEL = "wongsodillon/feedback-generator-falcon7b"

In [ ]:
model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)